In [1]:
import os
file_names:list[str] = os.listdir('stations')
abs_paths:list[str] = []
for name in file_names:
    if '每日各站進出站人數' in name:
        abs_paths.append(os.path.abspath(f'stations/{name}'))
    
abs_paths

['/workspaces/_2024_02_22/lesson22/stations/每日各站進出站人數2022.csv',
 '/workspaces/_2024_02_22/lesson22/stations/每日各站進出站人數2023.csv',
 '/workspaces/_2024_02_22/lesson22/stations/每日各站進出站人數20190423-20191231.csv',
 '/workspaces/_2024_02_22/lesson22/stations/每日各站進出站人數2021.csv',
 '/workspaces/_2024_02_22/lesson22/stations/每日各站進出站人數2020.csv']

In [2]:
import pandas as pd
from pandas import DataFrame

all_df:list[DataFrame] = [pd.read_csv(path) for path in abs_paths]
display(all_df)

[       trnOpDate  staCode  gateInComingCnt  gateOutGoingCnt
 0       20220101      900             8645             8526
 1       20220101      910              996             1243
 2       20220101      920             1369             1711
 3       20220101      930             3663             4535
 4       20220101      940             1532             1845
 ...          ...      ...              ...              ...
 86547   20221231     7360             3984             3893
 86548   20221231     7361               92              115
 86549   20221231     7362              131              159
 86550   20221231     7380              494              426
 86551   20221231     7390              313              274
 
 [86552 rows x 4 columns],
        trnOpDate  staCode  gateInComingCnt  gateOutGoingCnt
 0       20230101      900             8030             8241
 1       20230101      910             1017             1326
 2       20230101      920             1400             

In [3]:
inOut_df = pd.concat(all_df)
inOut_df.columns = ['乘車日','車站代碼','進站人數','出站人數']
inOut_df

,乘車日,車站代碼,進站人數,出站人數
0,20220101,900,8645,8526
1,20220101,910,996,1243
2,20220101,920,1369,1711
3,20220101,930,3663,4535
4,20220101,940,1532,1845
...,...,...,...,...
86927,20201231,7360,3523,3602
86928,20201231,7361,3,18
86929,20201231,7362,28,34
86930,20201231,7380,609,604


In [4]:
import requests
url2 = 'https://ods.railway.gov.tw/tra-ods-web/ods/download/dataResource/0518b833e8964d53bfea3f7691aea0ee'
response = requests.get(url2)
if response.status_code == 200:
    station_data = response.json()

station_df = pd.DataFrame(station_data,columns=['stationCode','stationName'])
station_df.columns = ['車站代碼','中文站名']
station_df['車站代碼'] = station_df['車站代碼'].astype(int)
station_df

,車站代碼,中文站名
0,900,基隆
1,910,三坑
2,920,八堵
3,930,七堵
4,940,百福
...,...,...
238,7360,瑞芳
239,7361,海科館
240,7362,八斗子
241,7380,四腳亭


In [5]:
station_all_df = pd.merge(inOut_df,station_df,on='車站代碼',how='left')
station_all_df

,乘車日,車站代碼,進站人數,出站人數,中文站名
0,20220101,900,8645,8526,基隆
1,20220101,910,996,1243,三坑
2,20220101,920,1369,1711,八堵
3,20220101,930,3663,4535,七堵
4,20220101,940,1532,1845,百福
...,...,...,...,...,...
406756,20201231,7360,3523,3602,瑞芳
406757,20201231,7361,3,18,海科館
406758,20201231,7362,28,34,八斗子
406759,20201231,7380,609,604,四腳亭


In [6]:
station_all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406761 entries, 0 to 406760
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   乘車日     406761 non-null  int64 
 1   車站代碼    406761 non-null  int64 
 2   進站人數    406761 non-null  int64 
 3   出站人數    406761 non-null  int64 
 4   中文站名    406761 non-null  object
dtypes: int64(4), object(1)
memory usage: 15.5+ MB


In [7]:
#移除Na
station_all_df1 = station_all_df.dropna()
station_all_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406761 entries, 0 to 406760
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   乘車日     406761 non-null  int64 
 1   車站代碼    406761 non-null  int64 
 2   進站人數    406761 non-null  int64 
 3   出站人數    406761 non-null  int64 
 4   中文站名    406761 non-null  object
dtypes: int64(4), object(1)
memory usage: 15.5+ MB


In [8]:
#乘車日變為datetime
station_all_df1['乘車日'] = pd.to_datetime(station_all_df1['乘車日'].astype(str))
station_all_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406761 entries, 0 to 406760
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   乘車日     406761 non-null  datetime64[ns]
 1   車站代碼    406761 non-null  int64         
 2   進站人數    406761 non-null  int64         
 3   出站人數    406761 non-null  int64         
 4   中文站名    406761 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 15.5+ MB


In [9]:
station_all_df2 = station_all_df1.sort_values(by='乘車日')
station_all_df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406761 entries, 173161 to 172948
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   乘車日     406761 non-null  datetime64[ns]
 1   車站代碼    406761 non-null  int64         
 2   進站人數    406761 non-null  int64         
 3   出站人數    406761 non-null  int64         
 4   中文站名    406761 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 18.6+ MB


In [10]:
station_all_df2['乘車日'].dt.strftime('%Y-%m')

173161    2019-04
173314    2019-04
173315    2019-04
173316    2019-04
173317    2019-04
           ...   
173021    2023-12
173022    2023-12
173023    2023-12
173011    2023-12
172948    2023-12
Name: 乘車日, Length: 406761, dtype: object

In [11]:
station_all_df2

,乘車日,車站代碼,進站人數,出站人數,中文站名
173161,2019-04-23,900,8442,7743,基隆
173314,2019-04-23,5080,15,14,鎮安
173315,2019-04-23,5090,368,325,林邊
173316,2019-04-23,5100,95,95,佳冬
173317,2019-04-23,5110,66,50,東海
...,...,...,...,...,...
173021,2023-12-31,3450,1063,869,林內
173022,2023-12-31,3460,325,269,石榴
173023,2023-12-31,3470,6964,6271,斗六
173011,2023-12-31,3400,693,677,永靖


In [14]:
a=station_all_df2.groupby('中文站名')[['進站人數','出站人數']].agg(['sum','max','min'])

In [15]:
a.unstack()
a.stack(["人數","統計"])

           中文站名
進站人數  sum  七堵      9044563
           三坑      2080517
           三塊厝      786111
           三姓橋     1760760
           三民        12535
                    ...   
出站人數  min  鼓山           16
           龍井           35
           龍泉            1
           龍港            2
           龜山            0
Length: 1446, dtype: int64